In [1]:
import collections
import evaluate

import pandas as pd
import numpy as np
import torch

from datasets import Dataset, DatasetDict
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, DefaultDataCollator, create_optimizer
from transformers.keras_callbacks import PushToHubCallback
from transformers import TrainingArguments, Trainer

device = "cuda:0" if torch.cuda.is_available() else "cpu"

2023-03-22 14:22:40.469961: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-22 14:22:41.150548: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/amazon/efa/lib:/opt/amazon/openmpi/lib:/usr/local/cuda/efa/lib:/usr/local/cuda/lib:/usr/local/cuda:/usr/local/cuda/lib64:/usr/local/cuda/extras/CUPTI/lib64:/usr/local/cuda/targets/x86_64-linux/lib:/usr/local/lib:/usr/lib:/opt/amazon/efa/lib:/opt/amazon/openmpi/lib:/usr/local/cuda/efa/lib:/usr/local/cuda/lib:/usr/local/cuda:/usr/local/cuda/lib64:/usr/local/cuda/extras/CUPTI/lib64:/usr/local/cud

## Load Data and remove external Question Answer pairs

In [2]:
df = pd.read_excel('../WOT_extractive_final.xlsx')

df.to_csv('temp.csv')
df = pd.read_csv('temp.csv').rename(columns={'Unnamed: 0': 'id'}).drop(columns=['Unnamed: 0.1'])
df['id'] = df['id'].astype(str)

df['output'] = df['output'].fillna('unanswerable')

# !!!!!!!!!!!!!!!!!!!!!!
# CHANGE CONTEXT
# df['input'] = df['input_context_selected']
# df['input'] = df.apply(lambda row: f"{row['question']}SPLITHistory: {row['history']}", axis=1)
# !!!!!!!!!!!!!!!!!!!!!!

df = df[~df['output'].isin(['yes', 'no', 'You are done'])]

df['question'] = df['input'].apply(lambda x: x.split('SPLIT')[0])
df['context'] = df['input'].apply(lambda x: x.split('SPLIT')[1])
df['answer'] = df['output']
df['answer_start'] = df.apply(lambda row: [i for i in range(len(row['context'])) if row['context'][i:i+len(row['answer'])] == row['answer']], axis=1)

df['answers'] = df.apply(lambda row: {'text': [row['answer']], 'answer_start': row['answer_start']} , axis=1)

df = df[['id', 'domain', 'labels', 'question', 'context', 'answers', 'data_split', 'is_factoid', 'is_confirmation', 'is_complex', 'is_causal', 'is_listing', 'is_history', 'is_navigation', 'is_unanswerable']].fillna('')

In [3]:
ds_train = Dataset.from_pandas(df[df['data_split']=='train'].reset_index())
ds_test = Dataset.from_pandas(df[df['data_split']=='test'].reset_index())
ds_val = Dataset.from_pandas(df[df['data_split']=='validation'].reset_index())

ds = DatasetDict({'train': ds_train,
                  'test': ds_test,
                  'validation': ds_val})
print(ds)

DatasetDict({
    train: Dataset({
        features: ['index', 'id', 'domain', 'labels', 'question', 'context', 'answers', 'data_split', 'is_factoid', 'is_confirmation', 'is_complex', 'is_causal', 'is_listing', 'is_history', 'is_navigation', 'is_unanswerable'],
        num_rows: 235
    })
    test: Dataset({
        features: ['index', 'id', 'domain', 'labels', 'question', 'context', 'answers', 'data_split', 'is_factoid', 'is_confirmation', 'is_complex', 'is_causal', 'is_listing', 'is_history', 'is_navigation', 'is_unanswerable'],
        num_rows: 397
    })
    validation: Dataset({
        features: ['index', 'id', 'domain', 'labels', 'question', 'context', 'answers', 'data_split', 'is_factoid', 'is_confirmation', 'is_complex', 'is_causal', 'is_listing', 'is_history', 'is_navigation', 'is_unanswerable'],
        num_rows: 163
    })
})


In [4]:
model_name = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

model = AutoModelForQuestionAnswering.from_pretrained(model_name).to(device)
model_dir = f"Models/{model_name}/Context_no_instructions/"

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForQuestionAnswering: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-cased and a

### Prepare Training Data

In [5]:
max_length = 384
stride = 128

def preprocess_training_examples(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    sample_map = inputs.pop("overflow_to_sample_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        sample_idx = sample_map[i]
        answer = answers[sample_idx]
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label is (0, 0)
        if offset[context_start][0] > start_char or offset[context_end][1] < end_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start a--nd end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

In [6]:
train_dataset = ds["train"].map(
    preprocess_training_examples,
    batched=True,
    remove_columns=ds["train"].column_names,
)
print(len(ds["train"]))
print(len(train_dataset))

A Jupyter Widget

235
1090


### Prepare Validation Data

In [7]:
def preprocess_validation_examples(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    sample_map = inputs.pop("overflow_to_sample_mapping")
    example_ids = []

    for i in range(len(inputs["input_ids"])):
        sample_idx = sample_map[i]
        example_ids.append(examples["id"][sample_idx])

        sequence_ids = inputs.sequence_ids(i)
        offset = inputs["offset_mapping"][i]
        inputs["offset_mapping"][i] = [
            o if sequence_ids[k] == 1 else None for k, o in enumerate(offset)
        ]

    inputs["example_id"] = example_ids
    return inputs

In [8]:
validation_dataset = ds["validation"].map(
    preprocess_validation_examples,
    batched=True,
    remove_columns=ds["validation"].column_names,
)

test_dataset = ds["test"].map(
    preprocess_validation_examples,
    batched=True,
    remove_columns=ds["test"].column_names,
)

A Jupyter Widget

A Jupyter Widget

### Evaluation Function

In [19]:
from tqdm.auto import tqdm

n_best = 20
max_answer_length = 128
predicted_answers = []
metric = evaluate.load("squad")

def compute_metrics(start_logits, end_logits, features, examples):
    example_to_features = collections.defaultdict(list)
    for idx, feature in enumerate(features):
        example_to_features[feature["example_id"]].append(idx)

    predicted_answers = []
    for example in tqdm(examples):
        example_id = example["id"]
        context = example["context"]
        answers = []

        # Loop through all features associated with that example
        for feature_index in example_to_features[example_id]:
            start_logit = start_logits[feature_index]
            end_logit = end_logits[feature_index]
            offsets = features[feature_index]["offset_mapping"]

            start_indexes = np.argsort(start_logit)[-1 : -n_best - 1 : -1].tolist()
            end_indexes = np.argsort(end_logit)[-1 : -n_best - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Skip answers that are not fully in the context
                    if offsets[start_index] is None or offsets[end_index] is None:
                        continue
                    # Skip answers with a length that is either < 0 or > max_answer_length
                    if (
                        end_index < start_index
                        or end_index - start_index + 1 > max_answer_length
                    ):
                        continue

                    answer = {
                        "text": context[offsets[start_index][0] : offsets[end_index][1]],
                        "logit_score": start_logit[start_index] + end_logit[end_index],
                    }
                    answers.append(answer)

        # Select the answer with the best score
        if len(answers) > 0:
            best_answer = max(answers, key=lambda x: x["logit_score"])
            predicted_answers.append(
                {"id": example_id, "prediction_text": best_answer["text"]}
            )
        else:
            predicted_answers.append({"id": example_id, "prediction_text": ""})

    theoretical_answers = [{"id": ex["id"], "answers": ex["answers"]} for ex in examples]

    metrics = metric.compute(predictions=predicted_answers, references=theoretical_answers)
        
    return metrics

## Training Arguments

In [20]:
per_device_eval_batch_size = 64     #64
per_device_train_batch_size = 16     #16
gradient_accumulation_steps = 4     # per_device_train_batch_size * gradient_accumulation_steps=64 change here to be 64

args = TrainingArguments(
    model_dir,
    evaluation_strategy="steps",
    eval_steps=100,
    logging_strategy="steps",
    logging_steps=100,
    save_strategy="steps",
    save_steps=100,
    learning_rate=4e-5,
    per_device_train_batch_size=per_device_train_batch_size,
    per_device_eval_batch_size=per_device_eval_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    weight_decay=0.01,
    fp16=True,
    max_steps=5000,
    optim='adafactor',
    label_names = ["start_positions", "end_positions"]
)

data_collator = DefaultDataCollator(return_tensors="pt")

def model_init():
    return AutoModelForQuestionAnswering.from_pretrained(model_name).to(device)


PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


## Fine Tune

In [21]:
trainer = Trainer(
    model_init=model_init,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer
)
trainer.train()

loading configuration file config.json from cache at /home/ubuntu/.cache/huggingface/hub/models--bert-base-cased/snapshots/5532cc56f74641d4bb33641f5c76a55d11f846e0/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.26.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 28996
}

loading weights file pytorch_model.bin from cache at /home/ubuntu/.cache/huggingface/hub/models--bert-base-cased/snapshots/5532cc56f74641d4bb33641f5c76a55d11f846e0/p

Step,Training Loss,Validation Loss


KeyboardInterrupt: 